In [ ]:
pip install pytesseract pillow transformers torch torchvision

<h1>1. Setup and Library Imports</h1>
This section handles mounting Google Drive (if running in Colab) and importing all necessary Python libraries.

In [ ]:
# Import Python libraries
import pytesseract
from PIL import Image
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from torchvision import transforms
import timm
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

<h1>2. Model Loading</h1>
Here, the pre-trained image and text classification models are loaded. The model file (ai_vs_human_ads_classifier_finetuned.pth) is accessible in Google Drive path.

In [ ]:
# Load image classifier
image_model = timm.create_model('resnet18', pretrained=True)
image_model.fc = torch.nn.Linear(image_model.fc.in_features, 2)
image_model.load_state_dict(torch.load('drive/MyDrive/ai_vs_human_ads_classifier_finetuned.pth'))
image_model.eval()

# Load text classifier
tokenizer = AutoTokenizer.from_pretrained("roberta-base-openai-detector")
text_model = AutoModelForSequenceClassification.from_pretrained("roberta-base-openai-detector")
text_model.eval()

# Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Global list to store results for batch analysis
analysis_results = []

<h1>3. Core Prediction Functions</h1>
These functions encapsulate the logic for image prediction, text classification, and text extraction. Each function is designed to be self-contained and return detailed results.

<h2>Image Prediction Function</h2>

In [ ]:
def predict_image(path):
    """Enhanced image prediction with detailed probabilities"""
    image = Image.open(path).convert("RGB")
    img_t = transform(image).unsqueeze(0)

    with torch.no_grad():
        outputs = image_model(img_t)
        probs = F.softmax(outputs, dim=1)
        pred = torch.argmax(probs, 1).item()

        # Get probabilities for both classes
        real_prob = probs[0][0].item()
        ai_prob = probs[0][1].item()

    return {
        'prediction': ['Real', 'AI-Generated'][pred],
        'confidence': probs[0][pred].item(),
        'real_probability': real_prob,
        'ai_probability': ai_prob,
        'raw_output': outputs[0].tolist()
    }

<h2>Text Classification Function</h2>

In [ ]:
def classify_text(text, threshold=0.8):
    """Enhanced text classification with detailed metrics"""
    if not text.strip():
        return {
            'prediction': 'No text detected',
            'confidence': 0.0,
            'human_probability': 0.0,
            'ai_probability': 0.0,
            'text_length': 0,
            'word_count': 0
        }

    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = text_model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred].item()

        # Get probabilities for both classes
        human_prob = probs[0][0].item()
        ai_prob = probs[0][1].item()

        label = ["Human-written", "AI-generated"][pred]
        if confidence < threshold:
            label = "Uncertain"

    return {
        'prediction': label,
        'confidence': confidence,
        'human_probability': human_prob,
        'ai_probability': ai_prob,
        'text_length': len(text),
        'word_count': len(text.split()),
        'raw_output': outputs.logits[0].tolist()
    }

<h2>Text Extraction Function</h2>

In [ ]:
def extract_text(image_path):
    """Enhanced text extraction with preprocessing metrics"""
    try:
        # Load image
        image = Image.open(image_path).convert("RGB")
        original_size = image.size

        # Resize
        image = image.resize((image.width * 2, image.height * 2), Image.LANCZOS)

        # Convert to grayscale
        gray = image.convert("L")

        # Apply OpenCV processing
        img_cv = np.array(gray)
        _, thresh = cv2.threshold(img_cv, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Convert back to PIL
        processed = Image.fromarray(thresh)

        # OCR with config
        config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(processed, config=config)

        return {
            'text': text.strip(),
            'original_image_size': original_size,
            'processed_image_size': processed.size,
            'character_count': len(text.strip()),
            'word_count': len(text.strip().split()) if text.strip() else 0
        }
    except Exception as e:
        return {
            'text': '',
            'original_image_size': (0, 0),
            'processed_image_size': (0, 0),
            'character_count': 0,
            'word_count': 0,
            'error': str(e)
        }

<h1>4. Composite Analysis and Reporting Functions</h1>
These functions tie together the image and text analyses, calculate a composite score, make a final decision, and handle result storage and reporting.


<h2>Composite Score Calculation</h2>

In [ ]:
def calculate_composite_score(img_results, txt_results):
    """Calculate a composite AI detection score"""
    # Weights for different components
    img_weight = 0.6
    txt_weight = 0.4

    # Get AI probabilities
    img_ai_prob = img_results['ai_probability']
    txt_ai_prob = txt_results['ai_probability'] if txt_results['prediction'] != 'No text detected' else 0.5

    # Calculate weighted score
    composite_score = (img_ai_prob * img_weight) + (txt_ai_prob * txt_weight)

    return composite_score

<h2>Full Analysis Function</h2>

In [ ]:
def full_analysis(image_path, ground_truth=None, save_results=True):
    # Image analysis
    img_results = predict_image(image_path)

    # Text extraction
    text_data = extract_text(image_path)

    # Text analysis
    txt_results = classify_text(text_data['text'])

    # Composite analysis
    composite_score = calculate_composite_score(img_results, txt_results)

    # Final decision with numerical thresholds
    if composite_score > 0.7:
        final_decision = "Highly Likely AI-Generated"
        confidence_level = "High"
    elif composite_score > 0.5:
        final_decision = "Likely AI-Generated"
        confidence_level = "Medium"
    elif composite_score > 0.3:
        final_decision = "Uncertain"
        confidence_level = "Low"
    else:
        final_decision = "Likely Human-Generated"
        confidence_level = "Medium-High"


    # Final label for comparison
    if 'AI' in final_decision:
        final_label = 'AI-Generated'
    elif 'Human' in final_decision:
        final_label = 'Human-Generated'
    else:
        final_label = 'Unknown'

    # Store results for batch analysis
    if save_results:
        result_entry = {
            'timestamp': datetime.now(),
            'image_path': image_path,
            'img_prediction': img_results['prediction'],
            'img_confidence': img_results['confidence'],
            'img_real_prob': img_results['real_probability'],
            'img_ai_prob': img_results['ai_probability'],
            'text_extracted': text_data['character_count'] > 0,
            'text_char_count': text_data['character_count'],
            'text_word_count': text_data['word_count'],
            'txt_prediction': txt_results['prediction'],
            'txt_confidence': txt_results['confidence'],
            'txt_human_prob': txt_results['human_probability'],
            'txt_ai_prob': txt_results['ai_probability'],
            'composite_score': composite_score,
            'final_decision': final_decision,
            'confidence_level': confidence_level,
            'ground_truth': ground_truth,
            'img_correct': img_results['prediction'] == ground_truth,
            'txt_correct': (txt_results['prediction'] == ground_truth) if text_data['character_count'] > 0 else None,
            'final_correct': final_label == ground_truth
        }
        analysis_results.append(result_entry)

    return result_entry

<h2>Batch Analysis Function</h2>

In [ ]:
def batch_analysis(image_entries):
    """Analyze multiple images and return comprehensive statistics"""
    print(f"\n🔄 BATCH ANALYSIS: Processing {len(image_entries)} images...")

    batch_results = []
    for entry in image_entries:
        try:
            # Extract path and ground_truth from dict, provide default if missing
            path = entry['path'] if isinstance(entry, dict) else entry
            ground_truth = entry.get('ground_truth', 'Unknown') if isinstance(entry, dict) else 'Unknown'

            result = full_analysis(path, ground_truth=ground_truth)
            batch_results.append(result)
        except Exception as e:
            print(f"Error processing {entry}: {e}")

    return batch_results

<h2>Statistics Report Generation</h2>

In [ ]:
def generate_statistics_report():
    """Generate comprehensive statistics from all analysis results"""
    if not analysis_results:
        print("No analysis results available. Run some analyses first.")
        return

    df = pd.DataFrame(analysis_results)

    print(f"\n📊 STATISTICS REPORT")
    print(f"{'='*50}")
    print(f"Total Images Analyzed: {len(df)}")
    print(f"Date Range: {df['timestamp'].min()} to {df['timestamp'].max()}")

    # Image Classification Stats
    print(f"\n📷 IMAGE CLASSIFICATION:")
    img_pred_counts = df['img_prediction'].value_counts()
    for pred, count in img_pred_counts.items():
        percentage = (count / len(df)) * 100
        print(f"   {pred}: {count} ({percentage:.1f}%)")

    print(f"   Average Image Confidence: {df['img_confidence'].mean():.4f}")
    print(f"   Average AI Probability: {df['img_ai_prob'].mean():.4f}")

    # Text Analysis Stats
    print(f"\n📝 TEXT ANALYSIS:")
    text_detected = df['text_extracted'].sum()
    print(f"   Images with Text: {text_detected} ({(text_detected/len(df)*100):.1f}%)")
    print(f"   Average Characters per Image: {df['text_char_count'].mean():.1f}")
    print(f"   Average Words per Image: {df['text_word_count'].mean():.1f}")

    # Text classification for images with text
    text_df = df[df['text_extracted']]
    if len(text_df) > 0:
        txt_pred_counts = text_df['txt_prediction'].value_counts()
        for pred, count in txt_pred_counts.items():
            percentage = (count / len(text_df)) * 100
            print(f"   Text {pred}: {count} ({percentage:.1f}%)")

    # Composite Scores
    print(f"\n🎯 COMPOSITE ANALYSIS:")
    print(f"   Average Composite Score: {df['composite_score'].mean():.4f}")
    print(f"   Median Composite Score: {df['composite_score'].median():.4f}")
    print(f"   Score Standard Deviation: {df['composite_score'].std():.4f}")

    # Final Decisions
    print(f"\n🏁 FINAL DECISIONS:")
    final_counts = df['final_decision'].value_counts()
    for decision, count in final_counts.items():
        percentage = (count / len(df)) * 100
        print(f"   {decision}: {count} ({percentage:.1f}%)")

    return df

<h2>Plotting Analysis Results</h2>

In [ ]:
def plot_analysis_results():
    """Create visualizations of the analysis results"""
    if not analysis_results:
        print("No analysis results available for plotting.")
        return

    df = pd.DataFrame(analysis_results)

    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # Plot 1: Composite Score Distribution
    axes[0, 0].hist(df['composite_score'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('Distribution of Composite AI Scores')
    axes[0, 0].set_xlabel('Composite Score')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].axvline(df['composite_score'].mean(), color='red', linestyle='--', label=f'Mean: {df["composite_score"].mean():.3f}')
    axes[0, 0].legend()

    # Plot 2: Image vs Text AI Probabilities
    axes[0, 1].scatter(df['img_ai_prob'], df['txt_ai_prob'], alpha=0.6)
    axes[0, 1].set_title('Image AI Prob vs Text AI Prob')
    axes[0, 1].set_xlabel('Image AI Probability')
    axes[0, 1].set_ylabel('Text AI Probability')
    axes[0, 1].plot([0, 1], [0, 1], 'r--', alpha=0.5)

    # Plot 3: Final Decision Counts
    decision_counts = df['final_decision'].value_counts()
    axes[1, 0].pie(decision_counts.values, labels=decision_counts.index, autopct='%1.1f%%')
    axes[1, 0].set_title('Final Decision Distribution')

    # Plot 4: Confidence vs Composite Score
    confidence_map = {'Low': 1, 'Medium': 2, 'Medium-High': 3, 'High': 4}
    df['confidence_numeric'] = df['confidence_level'].map(confidence_map)
    axes[1, 1].scatter(df['composite_score'], df['confidence_numeric'], alpha=0.6)
    axes[1, 1].set_title('Composite Score vs Confidence Level')
    axes[1, 1].set_xlabel('Composite Score')
    axes[1, 1].set_ylabel('Confidence Level')
    axes[1, 1].set_yticks([1, 2, 3, 4])
    axes[1, 1].set_yticklabels(['Low', 'Medium', 'Medium-High', 'High'])

    plt.tight_layout()
    plt.show()

<h2>Binary Classification Metrics</h2>

In [ ]:
def plot_binary_classification_metrics(df, true_col, pred_col, title_suffix, ground_truth_labels, pred_mapping=None):
    """
    Plots confusion matrix and prints classification report for binary classification.

    Args:
        df (pd.DataFrame): The DataFrame containing the analysis results.
        true_col (str): The name of the column containing the ground truth labels.
        pred_col (str): The name of the column containing the predicted labels.
        title_suffix (str): A suffix to add to the plot title (e.g., "Image Analysis").
        ground_truth_labels (list): A list of the two expected ground truth labels (e.g., ["Human-Generated", "AI-Generated"]).
        pred_mapping (dict, optional): A dictionary to map predicted labels to ground truth labels.
                                       Defaults to None, in which case direct comparison is used.
    """
    # Filter for valid ground truth entries
    valid_df = df[df[true_col].isin(ground_truth_labels)].copy()

    if valid_df.empty:
        print(f"\n❗ Not enough valid data for {title_suffix} confusion matrix and report.")
        return

    y_true = valid_df[true_col]
    y_pred_raw = valid_df[pred_col]

    # Apply mapping if provided
    if pred_mapping:
        y_pred = y_pred_raw.map(pred_mapping)
        # Filter out predictions that couldn't be mapped to the ground truth labels (e.g., 'Uncertain')
        combined_df = pd.DataFrame({'true': y_true, 'pred': y_pred}).dropna()
        y_true = combined_df['true']
        y_pred = combined_df['pred']
    else:
        # Filter out any predictions not directly matching ground truth labels
        combined_df = pd.DataFrame({'true': y_true, 'pred': y_pred_raw})
        combined_df = combined_df[combined_df['pred'].isin(ground_truth_labels)].dropna()
        y_true = combined_df['true']
        y_pred = combined_df['pred']

    if y_true.empty or y_pred.empty:
        print(f"\n❗ After filtering, not enough binary data for {title_suffix} confusion matrix and report.")
        return

    # Ensure consistent order for labels in confusion matrix
    display_labels = ground_truth_labels

    print(f"\n--- {title_suffix} Performance ---")
    print(f"\n📉 Confusion Matrix for {title_suffix}:")
    cm = confusion_matrix(y_true, y_pred, labels=display_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
    disp.plot(cmap="Blues", values_format='d')
    plt.title(f"Confusion Matrix: Ground Truth vs {title_suffix} Prediction")
    plt.show()

    print(f"\n📋 Classification Report for {title_suffix}:")
    print(classification_report(y_true, y_pred, labels=display_labels, target_names=display_labels, zero_division=0))


<h2>Export Results to CSV</h2>

In [ ]:
def export_results_to_csv(filename='ai_detection_results.csv'):
    """Export analysis results to CSV file"""
    if not analysis_results:
        print("No results to export.")
        return

    df = pd.DataFrame(analysis_results)
    df.to_csv(filename, index=False)
    print(f"Results exported to {filename}")
    return filename

<h2>5. Running the Analysis</h2>
This is the main execution block where you define your image paths, run the batch analysis, and then call the reporting and plotting functions.


<h2>5.1. Running analysis on full dataset (both human + ai generated images)</h2>

In [ ]:
import os

# Paths to folders (adjust if your folder names are different)
real_folder = "drive/MyDrive/real_img"
ai_folder = "drive/MyDrive/ai_img"

# Build labeled image list for batch processing
image_entries = []

# Add real images with ground truth label
for filename in os.listdir(real_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_entries.append({
            'path': os.path.join(real_folder, filename),
            'ground_truth': 'Human-Generated'
        })

# Add AI-generated images
for filename in os.listdir(ai_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_entries.append({
            'path': os.path.join(ai_folder, filename),
            'ground_truth': 'AI-Generated'
        })

# Now run batch analysis on all images
batch_results = batch_analysis(image_entries)

# Generate overall statistics report and plots
df = generate_statistics_report()
plot_analysis_results()


# Common ground truth labels for both analyses
GT_LABELS = ['Human-Generated', 'AI-Generated']

# 1. Performance for Image Analysis
# Map 'Real' prediction to 'Human-Generated' ground truth
image_pred_mapping = {'Real': 'Human-Generated', 'AI-Generated': 'AI-Generated'}
plot_binary_classification_metrics(
    df=df,
    true_col='ground_truth',
    pred_col='img_prediction',
    title_suffix='Image Analysis',
    ground_truth_labels=GT_LABELS,
    pred_mapping=image_pred_mapping
)

# 2. Performance for Text Analysis
# Filter for images where text was actually extracted for meaningful text analysis
text_analysis_df = df[df['text_extracted']].copy()
# Map 'Human-written' to 'Human-Generated' and 'AI-generated' to 'AI-Generated'
# 'Uncertain' will be filtered out by the plot_binary_classification_metrics function
text_pred_mapping = {'Human-written': 'Human-Generated', 'AI-generated': 'AI-Generated'}
plot_binary_classification_metrics(
    df=text_analysis_df,
    true_col='ground_truth',
    pred_col='txt_prediction',
    title_suffix='Text Analysis',
    ground_truth_labels=GT_LABELS,
    pred_mapping=text_pred_mapping
)

# 3. Performance for Final Composite Decision (original plot_confusion_and_classification logic)
# This uses the final_decision which is already normalized to 'AI-Generated' or 'Human-Generated'
# by the internal logic of plot_confusion_and_classification, but we can use our new function for consistency.
# For this, we'll need to create a temporary column that maps 'Likely Human-Generated', etc. to the binary labels
df['final_binary_decision'] = df['final_decision'].apply(
    lambda x: 'AI-Generated' if 'AI' in x else ('Human-Generated' if 'Human' in x else np.nan)
)
plot_binary_classification_metrics(
    df=df,
    true_col='ground_truth',
    pred_col='final_binary_decision',
    title_suffix='Final Composite Decision Analysis',
    ground_truth_labels=GT_LABELS
)


# Export results to CSV
export_results_to_csv("combined_results.csv")

<h2>5.2. Running analysis on Human-generated dataset</h2>

In [ ]:
import os

# Paths to folders
real_folder = "drive/MyDrive/real_img"
analysis_results = []
# Build labeled image list for batch processing
image_entries = []

# Add real images with ground truth label
for filename in os.listdir(real_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_entries.append({
            'path': os.path.join(real_folder, filename),
            'ground_truth': 'Human-Generated'
        })

# Now run batch analysis on all images
batch_results = batch_analysis(image_entries)

# Generate overall statistics report and plots
df = generate_statistics_report()
plot_analysis_results()


# Common ground truth labels for both analyses
GT_LABELS = ['Human-Generated', 'AI-Generated']

# 1. Performance for Image Analysis
# Map 'Real' prediction to 'Human-Generated' ground truth
image_pred_mapping = {'Real': 'Human-Generated', 'AI-Generated': 'AI-Generated'}
plot_binary_classification_metrics(
    df=df,
    true_col='ground_truth',
    pred_col='img_prediction',
    title_suffix='Image Analysis',
    ground_truth_labels=GT_LABELS,
    pred_mapping=image_pred_mapping
)

# 2. Performance for Text Analysis
# Filter for images where text was actually extracted for meaningful text analysis
text_analysis_df = df[df['text_extracted']].copy()
# Map 'Human-written' to 'Human-Generated' and 'AI-generated' to 'AI-Generated'
# 'Uncertain' will be filtered out by the plot_binary_classification_metrics function
text_pred_mapping = {'Human-written': 'Human-Generated', 'AI-generated': 'AI-Generated'}
plot_binary_classification_metrics(
    df=text_analysis_df,
    true_col='ground_truth',
    pred_col='txt_prediction',
    title_suffix='Text Analysis',
    ground_truth_labels=GT_LABELS,
    pred_mapping=text_pred_mapping
)

# 3. Performance for Final Composite Decision (original plot_confusion_and_classification logic)
# This uses the final_decision which is already normalized to 'AI-Generated' or 'Human-Generated'
# by the internal logic of plot_confusion_and_classification, but we can use our new function for consistency.
# For this, we'll need to create a temporary column that maps 'Likely Human-Generated', etc. to the binary labels
df['final_binary_decision'] = df['final_decision'].apply(
    lambda x: 'AI-Generated' if 'AI' in x else ('Human-Generated' if 'Human' in x else np.nan)
)
plot_binary_classification_metrics(
    df=df,
    true_col='ground_truth',
    pred_col='final_binary_decision',
    title_suffix='Final Composite Decision Analysis',
    ground_truth_labels=GT_LABELS
)


# Export results to CSV
export_results_to_csv("human_results.csv")

<h2>5.3. Running analysis on AI-generated dataset</h2>

In [ ]:
# Paths to folders
ai_folder = "drive/MyDrive/ai_img"

# Build labeled image list for batch processing
image_entries = []
analysis_results = []

# Add AI-generated images
for filename in os.listdir(ai_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_entries.append({
            'path': os.path.join(ai_folder, filename),
            'ground_truth': 'AI-Generated'
        })

# Now run batch analysis on all images
batch_results = batch_analysis(image_entries)

# Generate overall statistics report and plots
df = generate_statistics_report()
plot_analysis_results()


# Common ground truth labels for both analyses
GT_LABELS = ['Human-Generated', 'AI-Generated']

# 1. Performance for Image Analysis
# Map 'Real' prediction to 'Human-Generated' ground truth
image_pred_mapping = {'Real': 'Human-Generated', 'AI-Generated': 'AI-Generated'}
plot_binary_classification_metrics(
    df=df,
    true_col='ground_truth',
    pred_col='img_prediction',
    title_suffix='Image Analysis',
    ground_truth_labels=GT_LABELS,
    pred_mapping=image_pred_mapping
)

# 2. Performance for Text Analysis
# Filter for images where text was actually extracted for meaningful text analysis
text_analysis_df = df[df['text_extracted']].copy()
# Map 'Human-written' to 'Human-Generated' and 'AI-generated' to 'AI-Generated'
# 'Uncertain' will be filtered out by the plot_binary_classification_metrics function
text_pred_mapping = {'Human-written': 'Human-Generated', 'AI-generated': 'AI-Generated'}
plot_binary_classification_metrics(
    df=text_analysis_df,
    true_col='ground_truth',
    pred_col='txt_prediction',
    title_suffix='Text Analysis',
    ground_truth_labels=GT_LABELS,
    pred_mapping=text_pred_mapping
)

# 3. Performance for Final Composite Decision (original plot_confusion_and_classification logic)
# This uses the final_decision which is already normalized to 'AI-Generated' or 'Human-Generated'
# by the internal logic of plot_confusion_and_classification, but we can use our new function for consistency.
# For this, we'll need to create a temporary column that maps 'Likely Human-Generated', etc. to the binary labels
df['final_binary_decision'] = df['final_decision'].apply(
    lambda x: 'AI-Generated' if 'AI' in x else ('Human-Generated' if 'Human' in x else np.nan)
)
plot_binary_classification_metrics(
    df=df,
    true_col='ground_truth',
    pred_col='final_binary_decision',
    title_suffix='Final Composite Decision Analysis',
    ground_truth_labels=GT_LABELS
)


# Export results to CSV
export_results_to_csv("ai_results.csv")